# STA 141B: Homework 4
Winter 2018

## Student Information

After the colons (in the same line) please write just your first name, last name, and the 9 digit student ID number below.

First Name: Mitchell

Last Name: Layton

Student ID: 912307956

## Instructions

We use a script that extracts your answers by looking for cells in between the cells containing the exercise statements.  So you 

- MUST add cells in between the exercise statements and add answers within them and
- MUST NOT modify the existing cells, particularly not the problem statement

To make markdown, please switch the cell type to markdown (from code) - you can hit 'm' when you are in command mode - and use the markdown language.  For a brief tutorial see: https://daringfireball.net/projects/markdown/syntax


## Requesting freshness

In this portion, you'll combine the assignment 3 data set with nutrition data from the [USDA Food Composition Databases](https://ndb.nal.usda.gov/ndb/search/list). The CSV file `fresh.csv` contains the fresh fruits and vegetables data you extracted in assignment 3.

The USDA Food Composition Databases have a [documented](https://ndb.nal.usda.gov/ndb/doc/index) web API that returns data in JSON format . You need a key in order to use the API. Only 1000 requests are allowed per hour, so it would be a good idea to use [caching][requests_cache].

[Sign up for an API key here](https://api.data.gov/signup/). The key will work with any Data.gov API. You may need the key again later in the quarter, so make sure you save it.

These modules may be useful:

* [requests](http://docs.python-requests.org/en/master/user/quickstart/)
* [requests_cache][]
* [urlparse](https://docs.python.org/2/library/urlparse.html)
* [pandas](http://pandas.pydata.org/pandas-docs/stable/)

[requests_cache]: https://pypi.python.org/pypi/requests-cache

__Exercise 1.1__ (20 pts)

Read the [search request documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-SEARCH.md), then write a function called `ndb_search()` that makes a search request. The function should accept the search term as an argument. The function should return the search result items as a list (for 0 items, return an empty list).

Note that the search url is: `https://api.nal.usda.gov/ndb/search`

As an example, a search for `"quail eggs"` should return this list:

```python
[{u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  u'ndbno': u'45094707',
  u'offset': 0},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'L&W, QUAIL EGGS, UPC: 024072000256',
  u'ndbno': u'45094890',
  u'offset': 1},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  u'ndbno': u'45099560',
  u'offset': 2},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  u'ndbno': u'45169279',
  u'offset': 3},
 {u'ds': u'BL',
  u'group': u'Branded Food Products Database',
  u'name': u"D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  u'ndbno': u'45178254',
  u'offset': 4},
 {u'ds': u'SR',
  u'group': u'Dairy and Egg Products',
  u'name': u'Egg, quail, whole, fresh, raw',
  u'ndbno': u'01140',
  u'offset': 5}]
```

As usual, make sure you document and test your function.

In [327]:
import requests
import sqlite3
conn = sqlite3.connect(':memory:')
conn.isolation_level = None
curr = conn.cursor()

# UNCOMMENT 3 lines below for cache method, OR past API KEY into 'my_api_key' variable
# had to use "pip install --upgrade requests-cache to get the module
#import requests_cache
#requests_cache.install_cache('demo_cache')

my_api_key = 'qGHsT1sXfETbsOnmXQGHuqFwtucab2rrMntX2kb4'

def ndb_search(search_term, key):
    """
    Input 1: Search term for "q" parameter in API search
    Input 2: Owned API key for access
    
    Output: List of the "item" variables relevant to the search term
    """
    
    url = 'https://api.nal.usda.gov/ndb/search/'
    # can set the website url parameters using dictionary and requests.get will build them out
    p = {'format':'json', 'api_key':key, 'q':search_term, 'output':'basic', 'sort':'n'}
    r = requests.get(url, params = p)
    #print("\nURL: "+ r.url + "\n")
    
    #select from r.json() list where we only get the element, 'item' : [stuff we want]   
    data = r.json()
    item = data['list']['item']

    return(item)
    
    
ndb_search('quail eggs',my_api_key)

[{'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'BUDDHA, QUAIL EGGS IN BRINE, UPC: 761934535098',
  'ndbno': '45099560',
  'offset': 0},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'CHAOKOH, QUAIL EGG IN BRINE, UPC: 044738074186',
  'ndbno': '45094707',
  'offset': 1},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': "D'ARTAGNAN, QUAIL EGGS, UPC: 736622102630",
  'ndbno': '45178254',
  'offset': 2},
 {'ds': 'SR',
  'group': 'Dairy and Egg Products',
  'name': 'Egg, quail, whole, fresh, raw',
  'ndbno': '01140',
  'offset': 3},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'GRAN SABANA, QUAIL EGGS, UPC: 819140010103',
  'ndbno': '45169279',
  'offset': 4},
 {'ds': 'BL',
  'group': 'Branded Food Products Database',
  'name': 'L&W, QUAIL EGGS, UPC: 024072000256',
  'ndbno': '45094890',
  'offset': 5}]

__Exercise 1.2__ (20 pts)

Use your search function to get NDB numbers for the foods in the `fresh.csv` file. It's okay if you don't get an NDB number for every food, but try to come up with a strategy that gets most of them. Discuss your strategy in a short paragraph.

Hints:

* The foods are all raw and unbranded.
* You can test search terms with the [online search page](https://ndb.nal.usda.gov/ndb/search/list).
* You can convert the output of `ndb_search()` to a data frame with `pd.DataFrame()`.
* The string methods for [Python](https://docs.python.org/2/library/stdtypes.html#string-methods) and [Pandas](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary) are useful here. It's okay if you use _simple_ regular expressions in the Pandas methods, although this exercise can be solved without them.
* You can merge data frames that have a column in common with `pd.merge()`.

STRATEGY/METHOD:

The code below returns the fruits in the "fresh.csv" file with their accompanying NDB number for each instance of the raw and fresh unbranded foods. For example, in the file there contains oranges, which can come in many different variaties such as Valencia oranges, Navel oranges, Mandarin oranges, etc. So for each instance of the fruit name orange, it will give us many different NDB numbers for these different types. My code below does this for each fruit and/or vegetable. 

My goal was to try and extrapolate as many of these numbers for each different variety of fruit or vegetable as accurately as possible. The process involved recreating the food list in the "fresh.csv" file to get rid of the underscores just in case it would interfere with the search function. Next, I created a list with n elements for each list of objects in the API relating to one fruit or vegetable (for as many fruits/vegetables our search generates from the csv file). 

For the list called, "names", each element in names had 2 elements with its 1st element equal to a list of the stripped down row called "name" from the API search list. This name includes the name of the fruit or vegetable I was trying to find, along with any additional words the used for identifying (e.g. ['melons', 'cantaloupe', 'raw']). The 2nd element had the NBD number associated with these stripped down words list with the specific fruit or vegetable I was trying to identify.

Lastly, using this list of stripped down words, and all of the names of the fruits/veggies from "fresh.csv" file, I only selected those fruits/veggies in which the names from the csv file were absoultely in the stripped down words name. During this process I accounted for plural and singular instances because our list of stripped down words could include, ['java', 'plum', 'jambolan', 'raw'], where the matching word we want is 'plum', but our in our "fresh.csv" file it is listed as 'plums' so it would have missed this particular case without the use of the implementation of methods from the "inflect" package.

The only problem with this solving this problem was that the description of the "name" in the API list could include more than one fruit or vegetable name. For example, ['tangerines', 'mandarin', 'oranges', 'raw'], or ['tomatoes', 'orange', 'raw'] are examples where we have tangerines, and tomatoes, but also have other identifiers making it confusing for the search. My output will likely print the name of the first occurance of the name. So in this case the NDB number associated with 'tangerines' and 'tomatoes'.

In [62]:
import pandas as pd
import numpy as np
import os

# package for comparing between words for plural vs singular, or making plural or singular, etc.
# installed via: pip install -e git+https://github.com/pwdyson/inflect.py#egg=inflect
import inflect
p = inflect.engine()

# make sure "fresh.csv" file is in same Jupyter directory as this notebook
try:
    ind = os.listdir().index('fresh.csv')
    file = os.listdir()[ind]
except ValueError:
    print('\nERROR:\n    Make sure "fresh.csv" is in working directory.\n')
 
fresh = pd.read_csv(file)
fresh = pd.DataFrame(fresh)
food = fresh['food']


#------------------------------------------------------------------------------
def name_parser(api_list):
    """
    Input: 
        list format from the API with variables "ds", "name", "ndbno", etc.
    Returns:
        Formatted decomposition of the "name" variable in the list which comes with many identifiers in the sentences.
    """
    name = api_list['name'].lower()
    decomp = name.replace(",", "").replace("(","").replace(")","").replace("-"," ").split(" ")
    raw1 = name.find("upc") == -1
    raw2 = api_list['ds'] == 'SR'
    if raw1 & raw2:
        # uncheck the print() statement below to see the stripped down identifier names from the API list.
        #print(decomp)
        return(decomp)
#------------------------------------------------------------------------------

    
# separating underscores in food names (in excel file) for the search function
food_list = []
for i in food:
    if i.find("_") == -1:
        food_list.append(i)
    else:
        i = i.split('_')
        word = i[0] + " " + i[1]
        food_list.append(word)
   
#------------------------------------------------------------------------------

names = []
for food in food_list:
    try: # need try except for "kiwi" search since there are no occurances of fresh/raw kiwi
        x = ndb_search(food + ", raw",my_api_key)
        for i in x:
            n = name_parser(i)
            if n != None:
                names.append([n,i['ndbno']])
    except KeyError:
        continue
    
#------------------------------------------------------------------------------

lis = []
#Utilizing "inflect" package for checking singular and plural fruit name cases
for i in names:
    for j in food_list:
        # checks if our csv file food names in API list "name"
        if j in i[0]:
            #print(j + ": "+ str(i[1]))
            lis.append(j+","+i[1])
        # if not it checks the plural case of j
        elif ((j not in i[0]) & (p.singular_noun(j) == False)):
            j2 = p.plural_noun(j)
            if j2 in i[0]:
                #print(j2 + ": "+ str(i[1]))
                lis.append(j2+","+i[1])
        # if not it checks the singular case of j
        elif ((j not in i[0]) & (p.plural_noun(j)[-2:] == "ss")):
            j3 = p.singular_noun(j)
            if j3 in i[0]:
                lis.append(j3+","+i[1])
                
#------------------------------------------------------------------------------
                
df = []                
for i in lis:
    sp = i.split(",")
    food = p.plural_noun(sp[0])
    if food[-2:] == "ss":
        food = food[:-1]
    ndbno = sp[1]
    df.append([food,ndbno])

    
dataframe = pd.DataFrame(df, columns=['Food',"NDB_Number"])
dataframe = dataframe.sort_values(by=['Food']).reset_index().drop(columns=['index'])
dataframe

,Food,NDB_Number
0,apples,09502
1,apples,09503
2,apples,09501
3,apples,09500
4,apples,09004
5,apples,09005
6,apples,09006
7,apples,03154
8,apples,09504
9,apples,42266


__Exercise 1.3__ (20 pts)

Read the [food reports V2 documentation](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md), then write a function called `ndb_report()` that requests a _basic_ food report. The function should accept the NDB number as an argument and return the list of nutrients for the food.

Note that the report url is: `https://api.nal.usda.gov/ndb/V2/reports`

For example, for `"09279"` (raw plums) the first element of the returned list should be:

```python
{u'group': u'Proximates',
 u'measures': [{u'eqv': 165.0,
   u'eunit': u'g',
   u'label': u'cup, sliced',
   u'qty': 1.0,
   u'value': u'143.93'},
  {u'eqv': 66.0,
   u'eunit': u'g',
   u'label': u'fruit (2-1/8" dia)',
   u'qty': 1.0,
   u'value': u'57.57'},
  {u'eqv': 151.0,
   u'eunit': u'g',
   u'label': u'NLEA serving',
   u'qty': 1.0,
   u'value': u'131.72'}],
 u'name': u'Water',
 u'nutrient_id': u'255',
 u'unit': u'g',
 u'value': u'87.23'}
```

Be sure to document and test your function.

In [213]:
my_api_key = 'qGHsT1sXfETbsOnmXQGHuqFwtucab2rrMntX2kb4'

def ndbno_report(ndbno_list, report_type, file_format, key):
    """
    Input 1: A single string of numbers or a list of up to 50 NDB numbers
    Input 2: report type: (b) basic, (f) full, s (stats)
    Input 3: file format: json or xml
    Input 4: Owned API key for access
    
    Output: json format report ofof 1 or more fruits/veggies based on NDB numbers
    """
    
    url = 'https://api.nal.usda.gov/ndb/V2/reports/'
    # can set the website url parameters using dictionary and requests.get will build them out
    p = {'format':file_format, 'api_key':key, 'ndbno':ndbno_list, 'type': report_type}
    r = requests.get(url, params = p)
    #print("\nURL: "+ r.url + "\n")
    
    #select from r.json() list where we only get the element, 'item' : [stuff we want]   
    data = r.json()
    item = data['foods'][0]
    nutrients = item ['food']['nutrients']

    return(nutrients)

#check if 1st element of returned list from "raw plums" ndbno 09279 matches professor's example
ndbno_report("09279", 'b', 'json', my_api_key)[0]

{'derivation': 'NONE',
 'group': 'Proximates',
 'measures': [{'eqv': 165.0,
   'eunit': 'g',
   'label': 'cup, sliced',
   'qty': 1.0,
   'value': '143.93'},
  {'eqv': 66.0,
   'eunit': 'g',
   'label': 'fruit (2-1/8" dia)',
   'qty': 1.0,
   'value': '57.57'},
  {'eqv': 151.0,
   'eunit': 'g',
   'label': 'NLEA serving',
   'qty': 1.0,
   'value': '131.72'}],
 'name': 'Water',
 'nutrient_id': '255',
 'unit': 'g',
 'value': '87.23'}

__Exercise 1.4__ (20 pts) Which foods provide the best combination of price, yield, and nutrition? You can use kilocalories as a measure of "nutrition" here, but more a detailed analysis is better. Use plots to support your analysis.

For this problem, I was concerned with the many ways one could define nutritious. While price and yield are relatively straightforward varialbes in analysis, I had to come up with a definitive and quantitative scale for measuring nutrition in a fruit or vegetable. Given the data of the vitamins and minerals and other factos, we can create a quantitative scale of the best/worst fruit/vegetables in terms of overall value. 

"The role of bioavailability is important in establishing nutrient requirements and using those requirements in food labeling. The amount of a nutrient in a food that the body can actually use may vary depending on age and physiologic condition, such as pregnancy. Nutrient availability is also important in testing and marketing infant foods, nutritional supplements, and enteral formulas (for patients who can't digest solid foods). "

Lastly, let us note that these data values of the nutrients have unlisted variances and provide only average value of the nutritional content. So certain fruits/veggies in certain regions and growing conditions may provide higher or lower nutritional content of a particular vitamin or mineral than what our data shows.

(NOTE): There are so many labels for exact form of the fruit/veggie (e.g. 'label': 'cup, sliced' ... 'label': ''label': 'apricot' ... 'label': 'oz' ... etc, that it is too much to try to find every single fruit/veggies in the more raw and wholesome form. Intead I will use the average value of the nutrient before the 'measures' object list for each fruit/veggie.

(THEREFORE): We can get rid of the "measures" column in the dataframe.

Lets consider: 
Vitamins: A (RAE), B12, C, D (D2 + D3), E, K
Minerals: Calcium, Iron, Zinc, Magnesium

In [205]:
def clean_df(df):
    """
    Input: pandas dataframe of the nutrients list of particular fruit
    Returns: cleaned and correctly formatted dataframe for nutrient analysit ONLY, (price, weight below)
    """
    minerals = ['Calcium, Ca', 'Iron, Fe','Zinc, Zn','Magnesium, Mg']
    vitamins = ['Vitamin A, RAE','Vitamin B-12','Vitamin C, total ascorbic acid','Vitamin D (D2 + D3)','Vitamin E (alpha-tocopherol)','Vitamin K (phylloquinone)']
    both = minerals+vitamins

    name_check = []
    nutrient_names =  df.iloc[:,4]
    for i in nutrient_names:
        name_check.append(i)

    a = set(sorted(list(name_check)))
    b = set(sorted(list(both)))
    important_nutrients = b.intersection(a)

    new_nutrient_df = df.iloc[:,[0,5,2,4,7,6]].sort_values(by = ['name'])
    new_nutrient_df = new_nutrient_df.reset_index().drop(columns=['index'])
    new_nutrient_df.columns = ['Food','Nutrient ID','Group','Nutrient','Value (g)','Unit']
    new_nutrient_df = pd.DataFrame(new_nutrient_df).set_index('Nutrient')
    new_nutrient_df = new_nutrient_df.loc[important_nutrients]

    units = new_nutrient_df.iloc[:,4]
    new_units = []
    values = new_nutrient_df.iloc[:,3]
    new_values = []

    for i in range(len(units)):
        if units[i] == 'mg':
            new_values.append((round(float(values[i]) * 0.001,6)))
            new_units.append('g')
        elif units[i] == 'µg':
            new_values.append((round(float(values[i]) * 0.000001,9)))
            new_units.append('g')


    new_nutrient_df['Value (g)'] = new_values
    new_nutrient_df['Unit'] = new_units 
    new_nutrient_df = pd.DataFrame(new_nutrient_df).reset_index().set_index('Food')

    return(new_nutrient_df)

num_list = []
# use single variety fruits/veggies for faster comparison
ndbnumbers_list = pd.DataFrame(dataframe).drop_duplicates(['Food']).iloc[:,1]

for i in ndbnumbers_list:
    num_list.append(i)
print(num_list)
    
def last_funct(ndb_numbers):
    master = pd.DataFrame()
    d = ndb_numbers
    for i in d:
        nutri = ndbno_report(i, 'b', 'json', my_api_key)
        food_name = dataframe[dataframe.NDB_Number == i].iloc[0,0]

        temp = [k for k in nutri]

        nutrients = pd.DataFrame(temp)
        nutrients.insert(0, "Food_Names", food_name)
        c = clean_df(nutrients)

        master = master.append(c)
    return(master)

main_list = last_funct(num_list)

['09502', '09021', '11007', '11011', '09038', '09040', '35015', '09050', '11116', '09181', '09063', '11205', '09404', '09132', '09184', '11233', '09191', '11278', '11294', '09202', '09226', '09236', '09415', '09266', '09061', '09286', '03934', '11430', '09302', '09140', '09218', '09326']


In [206]:
main_list.head()

,Nutrient,Nutrient ID,Group,Value (g),Unit
Food,,,,,
apples,Vitamin K (phylloquinone),430,Vitamins,0.000003,g
apples,"Calcium, Ca",301,Minerals,0.005000,g
apples,"Zinc, Zn",309,Minerals,0.000040,g
apples,"Magnesium, Mg",304,Minerals,0.005000,g
apples,"Iron, Fe",303,Minerals,0.000150,g


In [294]:
pd.DataFrame(ndbno_report("09279", 'b', 'json', my_api_key))


    
def enery(ndb_numbers):
    master = pd.DataFrame()
    d = ndb_numbers
    for i in d:
        nutri = ndbno_report(i, 'b', 'json', my_api_key)
        food_name = dataframe[dataframe.NDB_Number == i].iloc[0,0]

        temp = [k for k in nutri]

        nutrients = pd.DataFrame(temp)
        nutrients.insert(0, "Food_Names", food_name)
        c = nutrients.iloc[:,[0,4,6,7]]
        c = c[c.name == 'Energy']

        master = master.append(c)
    return(master)

Energy = enery(num_list)

In [295]:
Energy

,Food_Names,name,unit,value
1,apples,Energy,kcal,58
1,apricots,Energy,kcal,48
1,artichokes,Energy,kcal,47
1,asparaguses,Energy,kcal,20
1,avocados,Energy,kcal,167
1,bananas,Energy,kcal,89
1,blackberries,Energy,kcal,52
1,blueberries,Energy,kcal,57
1,cabbages,Energy,kcal,13
1,cantaloupes,Energy,kcal,34


In [322]:
# print each single fruit/veggie with accompanying nutrients
copy = main_list.copy().reset_index()
food_items = list(copy.iloc[:,0])

#Distinct Foods
output = []
for x in food_items:
    if x not in output:
        output.append(x)
     
sumz = []
for i in range(len(output)):
    ea = copy[copy.Food == output[i]]
    sumz.append((round(ea.iloc[:4].sum(axis=0)[4],3)))

new = pd.DataFrame({'Food':output, 'vitamin/mineral totals':sumz})
new

,Food,vitamin/mineral totals
0,apples,0.010
1,apricots,0.013
2,artichokes,0.044
3,asparaguses,0.024
4,avocados,0.013
5,bananas,0.005
6,blackberries,0.009
7,blueberries,0.006
8,cabbages,0.105
9,cantaloupes,0.009


In [320]:
import plotnine as gg
from plotnine.data import *

result

,vitamin/mineral totals,Food_Names,name,unit,value,vitamin/mineral totals
1,0.013,apples,Energy,kcal,58,0.013
1,0.013,apricots,Energy,kcal,48,0.013
1,0.013,artichokes,Energy,kcal,47,0.013
1,0.013,asparaguses,Energy,kcal,20,0.013
1,0.013,avocados,Energy,kcal,167,0.013
1,0.013,bananas,Energy,kcal,89,0.013
1,0.013,blackberries,Energy,kcal,52,0.013
1,0.013,blueberries,Energy,kcal,57,0.013
1,0.013,cabbages,Energy,kcal,13,0.013
1,0.013,cantaloupes,Energy,kcal,34,0.013
